In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import sklearn
import pickle as pkl

In [3]:
ipl=pd.read_csv('matches.csv')

In [4]:
ipl.shape

(756, 18)

In [5]:
# these are the parameters used to as X
new_ipl = ipl[['team1','team2','toss_decision','toss_winner','winner']]

In [6]:
# remove the empty dataset 
new_ipl=new_ipl.dropna(inplace=False)

In [7]:
new_ipl

,team1,team2,toss_decision,toss_winner,winner
0,Sunrisers Hyderabad,Royal Challengers Bangalore,field,Royal Challengers Bangalore,Sunrisers Hyderabad
1,Mumbai Indians,Rising Pune Supergiant,field,Rising Pune Supergiant,Rising Pune Supergiant
2,Gujarat Lions,Kolkata Knight Riders,field,Kolkata Knight Riders,Kolkata Knight Riders
3,Rising Pune Supergiant,Kings XI Punjab,field,Kings XI Punjab,Kings XI Punjab
4,Royal Challengers Bangalore,Delhi Daredevils,bat,Royal Challengers Bangalore,Royal Challengers Bangalore
...,...,...,...,...,...
751,Kolkata Knight Riders,Mumbai Indians,field,Mumbai Indians,Mumbai Indians
752,Chennai Super Kings,Mumbai Indians,bat,Chennai Super Kings,Mumbai Indians
753,Sunrisers Hyderabad,Delhi Capitals,field,Delhi Capitals,Delhi Capitals
754,Delhi Capitals,Chennai Super Kings,field,Chennai Super Kings,Chennai Super Kings


In [7]:
all_teams={}
cnt=0
x=len(ipl)
for i in range(x):
    if ipl.loc[i]['team1'] not in all_teams:
        all_teams[ipl.loc[i]['team1']]=cnt
        cnt+=1
    if ipl.loc[i]['team2'] not in all_teams:
        all_teams[ipl.loc[i]['team2']]=cnt
        cnt+=1

In [8]:
x=new_ipl[['team1','team2','toss_decision','toss_winner']]
y=new_ipl[['winner']]

In [9]:
encoded_teams={w:k for k,w in all_teams.items()}

In [10]:
X=np.array(x)
Y=np.array(y)
# print(X,Y)
print(all_teams)
# print(X[0][0],all_teams[X[0][0]])
for i in range(len(X)):
    X[i][0]=all_teams[X[i][0]]
    X[i][1]=all_teams[X[i][1]]
    X[i][3]=all_teams[X[i][3]]
    Y[i][0]=all_teams[Y[i][0]]

{'Sunrisers Hyderabad': 0, 'Royal Challengers Bangalore': 1, 'Mumbai Indians': 2, 'Rising Pune Supergiant': 3, 'Gujarat Lions': 4, 'Kolkata Knight Riders': 5, 'Kings XI Punjab': 6, 'Delhi Daredevils': 7, 'Chennai Super Kings': 8, 'Rajasthan Royals': 9, 'Deccan Chargers': 10, 'Kochi Tuskers Kerala': 11, 'Pune Warriors': 12, 'Rising Pune Supergiants': 13, 'Delhi Capitals': 14}


In [11]:
fb={'field':0,'bat':1}
# changing the toss decision to 0 and 1
for i in range(len(X)):
    X[i][2]=fb[X[i][2]]
    
#changing the  toss winner to 0 and 1
for i in range(len(X)):
    if X[i][3] ==X[i][0]:
        X[i][3]=0
    else:
        X[i][3]=1
    

In [12]:
ones=0
for i in range(len(Y)):
    if Y[i][0]==X[i][1]:
        if ones<370:
            ones+=1
            Y[i][0]=1
        else:
                X[i][0],X[i][1]=X[i][1],X[i][0]
                Y[i][0]=0
    else:
        Y[i][0]=0

In [13]:
X=np.array(X,dtype='int32')
# print(type(X[0][0]))
Y=np.array(Y,dtype='int32')
Y=Y.ravel()

In [14]:
print(np.unique(Y,return_counts=True))

(array([0, 1]), array([382, 370], dtype=int64))


In [15]:
from sklearn.model_selection import train_test_split
x_train_set,x_test_set,y_train_set,y_test_set = train_test_split(X,Y,test_size=0.2)

In [16]:
print(len(x_train_set),len(x_test_set),len(y_train_set),len(y_test_set))
print(x_train_set)

601 151 601 151
[[ 6  7  0  1]
 [ 1  4  1  0]
 [ 6  0  0  1]
 ...
 [ 1 12  0  1]
 [ 2  1  1  0]
 [ 2  0  0  1]]


In [17]:
from sklearn.ensemble import RandomForestClassifier
model3= RandomForestClassifier(n_estimators=250).fit(x_train_set,y_train_set)
model3.score(x_test_set,y_test_set)

0.5894039735099338

In [18]:
from sklearn.svm import SVC
model1=SVC().fit(x_train_set,y_train_set)
model1.score(x_train_set,y_train_set)

0.5740432612312812

In [19]:
from sklearn.tree import DecisionTreeClassifier
model2=DecisionTreeClassifier().fit(x_train_set,y_train_set)
model2.score(x_train_set,y_train_set)

0.7371048252911814

In [20]:
test=np.array([0,1,0,0]).reshape(1,-1)
model2.predict(test)

array([1])

In [21]:
import pickle as pkl
with open('model.pkl','wb') as f:
    pkl.dump(model2,f)

In [22]:
with open('vocab.pkl','wb') as f:
    pkl.dump(encoded_teams,f)
with open('inv_vocab.pkl','wb') as f:
    pkl.dump(all_teams,f)